In [1]:
import pyspark

spark = (
    pyspark.sql.SparkSession.builder
    .appName("Row Level Lineage")
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/10 22:45:36 WARN Utils: Your hostname, FRL-1SFXW94, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/09/10 22:45:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/10 22:45:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df1 = spark.createDataFrame(
    [
        ("A", 1), 
        ("A", 2),
        ("B", 3), 
        ("B", 4),
        ("B", 5), 
        ("C", 6),
    ],
    ["key", "value1"],
)
df2 = spark.createDataFrame(
    [
        ("A", 10),
        ("B", -3), 
        ("B", 2), 
        ("C", -6),
        ("C", -8),
        ("C", 9),
        ("D", 10),
        ("D", 4),
    ],
    ["key", "value2"],
)
df3 = spark.createDataFrame(
    [
        ("A", 5.3), 
        ("B", -4.1),
        ("C", 0.), 
        ("D", 7.6),
    ],
    ["key", "value3"],
)

In [8]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from pyspark.sql.window import Window

def pipeline(df1: DataFrame, df2: DataFrame, df3: DataFrame):
    # return (
    #     df1
    #     .groupBy("key")
    #     .agg(F.sum("value1").alias("value1"))
    # )
    return (
        df2
        .filter("key in ('B', 'D')")
        .withColumn("value2", F.mean("value2").over(Window.partitionBy("key")))
    )
    # return (
    #     df3
    #     .join(
    #         df2
    #         .filter("key in ('B', 'D')")
    #         .withColumn("value2", F.mean("value2").over(Window.partitionBy("key"))), 
    #         on="key", 
    #         how="inner"
    #     )
    #     .join(
    #         df1
    #         .groupBy("key")
    #         .agg(F.sum("value1").alias("value1")), 
    #         on="key", 
    #         how="inner"
    #     )
    # )

pipeline(df1, df2, df3).toPandas()

,key,value2
0,B,-0.5
1,B,-0.5
2,D,7.0
3,D,7.0


In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
from pyspark_row_lineage import row_lineage

with row_lineage(df1, df2, df3) as (df1_tmp, df2_tmp, df3_tmp):
    output_df = pipeline(df1_tmp, df2_tmp, df3_tmp)
    # df1_lineage = output_df.blame(df1_tmp)

output_df.toPandas()

{"ts": "2025-09-10 22:54:26.001", "level": "ERROR", "logger": "SQLQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `__row_ids_133325885898048` cannot be resolved. Did you mean one of the following? [`__row_ids_133325040655904`, `value2`, `key`]. SQLSTATE: 42703", "context": {"errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o522.withColumn.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `__row_ids_133325885898048` cannot be resolved. Did you mean one of the following? [`__row_ids_133325040655904`, `value2`, `key`]. SQLSTATE: 42703; line 1 pos 0;\n'Project [key#2, value2#76, __row_ids_133325040655904#74, '__row_ids_133325885898048 AS __row_ids_133325885898048#79]\n+- Project [key#2, value2#76, __row_ids_133325040655904#74]\n   +- Project [key#2, __row_ids_1

toto
Column<'avg(value2) OVER (PARTITION BY key)'>
toto


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `__row_ids_133325885898048` cannot be resolved. Did you mean one of the following? [`__row_ids_133325040655904`, `value2`, `key`]. SQLSTATE: 42703; line 1 pos 0;
'Project [key#2, value2#76, __row_ids_133325040655904#74, '__row_ids_133325885898048 AS __row_ids_133325885898048#79]
+- Project [key#2, value2#76, __row_ids_133325040655904#74]
   +- Project [key#2, __row_ids_133325040655904#74, value2#3L, value2#76, value2#76]
      +- Window [avg(value2#3L) windowspecdefinition(key#2, specifiedwindowframe(RowFrame, unboundedpreceding$(), unboundedfollowing$())) AS value2#76], [key#2]
         +- Project [key#2, __row_ids_133325040655904#74, value2#3L]
            +- Filter key#2 IN (B,D)
               +- Project [key#2, value2#3L, array(monotonically_increasing_id()) AS __row_ids_133325040655904#74]
                  +- LogicalRDD [key#2, value2#3L], false


In [ ]:
from pyspark.sql.classic.dataframe import DataFrame
DataFrame.select = lambda x: x

In [28]:
df1_tmp.join

<bound method DataFrame.join of DataFrame[key: string, value1: bigint, __row_ids_133325885898048: array<bigint>]>

In [11]:
df2_tmp.toPandas()

,key,value2,__row_ids_133325040655904
0,A,10,[17179869184]
1,B,-3,[42949672960]
2,B,2,[68719476736]
3,C,-6,[85899345920]
4,C,-8,[111669149696]
5,C,9,[137438953472]
6,D,10,[163208757248]
7,D,4,[180388626432]


In [7]:
df2_tmp.toPandas()

,key,value2,__row_ids_133325040655904
0,A,10,[17179869184]
1,B,-3,[42949672960]
2,B,2,[68719476736]
3,C,-6,[85899345920]
4,C,-8,[111669149696]
5,C,9,[137438953472]
6,D,10,[163208757248]
7,D,4,[180388626432]
